# Data transformation

This model will predict whether a default will happen in the next twelve months.

In [1]:
import random

import pandas as pd
import numpy as np

path_to_csv = "../data/mortgage_sample.csv"
df = pd.read_csv(path_to_csv)

In [3]:
df.loc[df["id"] == 1]

,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,investor_orig_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time,payoff_time,status_time,sample
0,1,25,-7,25,113,41303.42,24.498336,9.2,226.29,2.899137,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
1,1,26,-7,25,113,41061.95,24.483867,9.2,225.10,2.151365,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
2,1,27,-7,25,113,40804.42,24.626795,9.2,222.39,2.361722,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
3,1,28,-7,25,113,40483.89,24.735883,9.2,219.67,1.229172,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
4,1,29,-7,25,113,40367.06,24.925476,9.2,217.37,1.692969,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
5,1,30,-7,25,113,40127.97,25.318291,9.2,212.73,2.274218,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
6,1,31,-7,25,113,39718.66,26.566117,9.2,200.67,1.850689,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
7,1,32,-7,25,113,35877.03,25.872559,9.2,186.12,1.104163,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
8,1,33,-7,25,113,34410.03,25.584425,9.2,180.52,0.836859,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public
9,1,34,-7,25,113,33590.47,26.008073,9.2,173.35,-0.314448,...,0,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public


Looking at customer with the first index, we can see that they default after 23 months. We will have to transform the dataframe, so that we have rows that contain twelve months of information. 

This will be achieved by creating dataframes for each id. From those dataframes, twelve month windows will be made into singular rows, for which the result of the loan will be known.

For example, rows 12 to 23 may be selected. From this range, values from a random row will be made into a new one, and the final result will be added as a column.

The new row will be added to a new dataset. From this dataset, we will split the data for training and testing. Later, a cross-validation may be implemented

In [146]:
# Create a subset for a specified ID
def create_subset(df, borrower_id):
    subset_df = df.loc[df["id"] == borrower_id]
    return subset_df

# Split subset into twelve-month long windows (or less)
def split_subset(subset_df, window_length=12):
    subset_len = subset_df.shape[0]
    
    windows = list()
    
    # TODO find a more clever way to do this than using a while loop
    current = 0
    while current < subset_len:
        window = subset_df.iloc[current:current+window_length if current+window_length < subset_len else subset_len]
        windows.append(window)
        current += 12
        
    return windows
        
def create_usable_row(window_df, include_result=True):
    window_len = window_df.shape[0]
    if window_len > 2:
    
        chosen_row = window_df.iloc[[random.randint(0, window_len-2)]].to_dict()
        last_row = window_df.iloc[[window_len-1]].to_dict()

        # We will use the same methodology as the original dataset
        # Meaning 1 is a default, 2 is a payoff and 0 means nothing changed
        if include_result:
            chosen_row["result"] = {
                list(chosen_row["id"].keys())[0]:
                list(last_row["status_time"].values())[0]
            }

        # return [list(dictionary.values())[0] for dictionary in list(chosen_row.values())]
        return pd.DataFrame(chosen_row)

Let's see how well it works by selecting the first customer

In [142]:
subset_df = create_subset(df=df, borrower_id=1)
windows = split_subset(subset_df=subset_df)
for window_df in windows:
    print(create_usable_row(window_df=window_df))

    id  time  orig_time  first_time  mat_time  balance_time   LTV_time  \
10   1    35         -7          25       113      32952.48  27.286499   

    interest_rate_time  hpi_time  gdp_time  ...  balance_orig_time  \
10                 9.2    162.09 -2.805844  ...            45000.0   

    FICO_orig_time  LTV_orig_time  Interest_Rate_orig_time  hpi_orig_time  \
10             715           69.4                      9.2          87.03   

    default_time  payoff_time  status_time  sample  result  
10           0.0          0.0          0.0  public     0.0  

[1 rows x 25 columns]
    id  time  orig_time  first_time  mat_time  balance_time   LTV_time  \
19   1    44         -7          25       113      30585.43  27.202731   

    interest_rate_time  hpi_time  gdp_time  ...  balance_orig_time  \
19                 9.2    150.91  1.875515  ...            45000.0   

    FICO_orig_time  LTV_orig_time  Interest_Rate_orig_time  hpi_orig_time  \
19             715           69.4          

This seems to work fine. We can turn extract the data from the main dataframe and create a new dataframe which will be usable for our XGBoost model. We will still drop the rows with NaN values though.

In [28]:
df = df.dropna(axis=0)

In [147]:
unique_ids = list(df.id.unique())

output_df = pd.DataFrame()

for borrower_id in unique_ids:
    subset_df = create_subset(df=df, borrower_id=borrower_id)
    windows = split_subset(subset_df=subset_df)
    for window_df in windows:
        usable_df = create_usable_row(window_df=window_df)
        
        # output_df.loc[-1] = usable_df
        # output_df.reset_index(drop=True)
        output_df = output_df.append(usable_df, ignore_index=True)

C:\Users\vojte\AppData\Local\Temp\ipykernel_10036\4094305794.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(usable_df, ignore_index=True)


In [148]:
output_df.head()

,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time,payoff_time,status_time,sample,result
0,1,32,-7,25,113,35877.03,25.872559,9.200,186.12,1.104163,...,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public,0.0
1,1,40,-7,25,113,32388.30,27.828850,9.200,156.21,1.585966,...,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public,1.0
2,4,28,-2,25,119,60576.14,34.672545,10.875,219.67,1.229172,...,63750.0,587,81.8,10.5,97.99,0.0,0.0,0.0,public,0.0
3,4,42,-2,25,119,57950.27,45.256864,9.250,161.00,3.029499,...,63750.0,587,81.8,10.5,97.99,0.0,0.0,0.0,public,0.0
4,4,52,-2,25,119,55332.84,43.177891,10.500,161.13,1.081049,...,63750.0,587,81.8,10.5,97.99,0.0,0.0,0.0,public,0.0


This seems to work fine. Let's do it for all of the data and output it as csv.

In [149]:
path_to_output = "../data/mortgage_transformed.csv"
output_df.to_csv(path_to_output)